In [ ]:
import numpy as np
import pandas as pd
import torch
from transformers import T5Tokenizer, T5ForConditionalGeneration, AdamW
from torch.utils.data import DataLoader, Dataset

In [ ]:
import torch
torch.cuda.is_available()


In [ ]:
# Đọc dữ liệu từ tập train của Spider dataset
df2 = pd.read_csv('data/spider/spider_schema_train.csv')  # Đảm bảo đường dẫn đúng


In [ ]:
# Tạo danh sách câu hỏi và câu lệnh SQL
questions = df2['question'].tolist()
sql_queries = df2['sql'].tolist()

In [ ]:
sql_queries

In [ ]:
# Tokenizer từ T5
tokenizer = T5Tokenizer.from_pretrained('t5-base')
tokenized_inputs = tokenizer.batch_encode_plus(questions, padding=True, truncation=True, return_tensors='pt')
tokenized_outputs = tokenizer.batch_encode_plus(sql_queries, padding=True, truncation=True, return_tensors='pt')

In [ ]:
# Dataset tùy chỉnh cho SQL
class SQLOnlineDataset(Dataset):
    def __init__(self, tokenized_inputs, tokenized_outputs):
        self.input_ids = tokenized_inputs['input_ids']
        self.attention_mask = tokenized_inputs['attention_mask']
        self.labels = tokenized_outputs['input_ids']
        self.decoder_attention_mask = tokenized_outputs['attention_mask']

    def __len__(self):
        return len(self.input_ids)

    def __getitem__(self, idx):
        return {
            'input_ids': self.input_ids[idx],
            'attention_mask': self.attention_mask[idx],
            'labels': self.labels[idx],
            'decoder_attention_mask': self.decoder_attention_mask[idx]
        }

In [ ]:
# Tạo DataLoader cho tập train
train_dataset = SQLOnlineDataset(tokenized_inputs, tokenized_outputs)
train_loader = DataLoader(train_dataset, batch_size=4, shuffle=True)

In [ ]:
# Sử dụng GPU nếu có
device = torch.device('cpu')
model = T5ForConditionalGeneration.from_pretrained('t5-base').to(device)

In [ ]:
device


In [ ]:
# Cấu hình optimizer
optimizer = AdamW(model.parameters(), lr=3e-5)

In [ ]:
# Huấn luyện mô hình
num_epochs = 10
for epoch in range(num_epochs):
    for batch in train_loader:
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)
        decoder_attention_mask = batch['decoder_attention_mask'].to(device)
        optimizer.zero_grad()
        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels,
            decoder_attention_mask=decoder_attention_mask,
            return_dict=True
        )
        loss = outputs.loss
        loss.backward()
        optimizer.step()
    print(f'Epoch: {epoch+1}, Loss: {loss.item()}')

In [ ]:
# Lưu mô hình và tokenizer
model.save_pretrained('model/spider/spider_t5_model')
tokenizer.save_pretrained('model/spider/spider_t5_tokenizer')

In [ ]:
torch.save(model.state_dict(), 'model/spider/spider_model.pt')

In [ ]:
# Load lại mô hình và tokenizer để kiểm tra
model = T5ForConditionalGeneration.from_pretrained('t5-base')
model.load_state_dict(torch.load('model/spider/spider_model.pt'))
model.eval()

In [ ]:
torch.cuda.is_available()

In [ ]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')

In [ ]:
# Thử nghiệm với câu hỏi mới
new_question = "What are the first and last names of all customers with more than 2 payments?"
input_ids = tokenizer.encode(new_question, return_tensors='pt')
outputs = model.generate(input_ids=input_ids, max_length=100, num_beams=5, early_stopping=True)
sql_query = tokenizer.decode(outputs[0], skip_special_tokens=True)

print(f"Question: {new_question}")
print(f"Generated SQL query: {sql_query}")